In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

Chấm điểm

In [ ]:
df = pd.read_csv('./dataset/error/angles.csv')
df = df.drop('Unnamed: 0', axis=1)
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)
df['Score'] = df['Score'].astype(int)
df.columns = df.columns.astype(str)
df

LR

In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)
Ypred = reg.predict(Xtest)
Ypred = Ypred.astype(int)

In [ ]:
Ypred

In [ ]:
Ytest

In [ ]:
# Đánh giá độ chính xác
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

SVR

In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)
Ypred = svr.predict(Xtest)
Ypred = Ypred.astype(int)

In [ ]:
Ypred

In [ ]:
Ytest

In [ ]:
# Đánh giá độ chính xác
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

TensorFlow

In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Chuẩn hoá dữ liệu
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [ ]:
# Xây dựng mô hình mạng neural
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(9,)),  # Số lượng đặc trưng đầu vào
    tf.keras.layers.Dense(64, activation='relu'),  # Lớp ẩn với 64 đơn vị ẩn
    tf.keras.layers.Dense(32, activation='relu'),  # Lớp ẩn với 32 đơn vị ẩn
    tf.keras.layers.Dense(1)  # Lớp đầu ra với 1 đơn vị (Regression)
])

In [ ]:
# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(Xtrain, Ytrain, epochs=50, batch_size=32, validation_data=(Xtest, Ytest))

# Đánh giá hiệu suất của mô hình
loss = model.evaluate(Xtest, Ytest)
print(f"Loss on test data: {loss}")

In [ ]:
# Đánh giá độ chính xác
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

Random Forest

In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
# Tạo mô hình Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Bạn có thể tùy chỉnh số cây và các siêu tham số khác

# Huấn luyện mô hình trên tập huấn luyện
rf_model.fit(Xtrain, Ytrain)

# Dự đoán trên tập kiểm tra
Ypred = rf_model.predict(Xtest)

# Đánh giá mô hình
mse = mean_squared_error(Ytest, Ypred)
r2 = r2_score(Ytest, Ypred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

In [ ]:
# Đánh giá độ chính xác
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')